In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install bing-image-downloader gradio tensorflow


In [ ]:
from bing_image_downloader import downloader

classes_money = ["1000 VND", "2000 VND", "5000 VND", "10000 VND", "20000 VND",
                 "50000 VND", "100000 VND", "200000 VND", "500000 VND"]

for c in classes_money:
    downloader.download(c, limit=40, output_dir='money_dataset', adult_filter_off=True, force_replace=False, timeout=60)


[%] Downloading Images to /content/money_dataset/1000 VND


[!!]Indexing page: 1

[%] Indexed 35 Images on Page 1.


[%] Downloading Image #1 from http://numismaclub.com/imgs/a/i/p/q/b/vietnamese_dong_1000_uncirculated_banknote_vnd_currency_vietnam_money_1000_1_lgw.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://scootersaigontour.com/wp-content/uploads/2020/04/vnd-1000-banknote-guide-to-Vietnamese-currency.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://watermark.lovepik.com/photo/20211119/large/lovepik-1000-vietnam-shield-picture_500186406.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://evivatour.com/wp-content/uploads/2022/07/1000vnd.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://c8.alamy.com/comp/2BTRFAR/thousand-dongs-vietnamese-ng-vietnamesischer-dong-vnd-vietnam-asia-2BTRFAR.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://www.leftovercurrency.com/app/uploads/2019/04/1000-vietnamese-dong-banknote

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import pickle

img_size = (128,128)

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    "money_dataset",
    target_size=img_size,
    batch_size=16,
    subset="training",
    class_mode="categorical"
)

val_gen = datagen.flow_from_directory(
    "money_dataset",
    target_size=img_size,
    batch_size=16,
    subset="validation",
    class_mode="categorical"
)

model_money = models.Sequential([
    layers.Flatten(input_shape=(128,128,3)),
    layers.Dense(512, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(len(classes_money), activation="softmax")
])

model_money.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

history = model_money.fit(train_gen, validation_data=val_gen, epochs=10)

# Lưu model và labels
model_money.save("/content/drive/MyDrive/money_ann_best.h5")
with open("/content/drive/MyDrive/money_labels.pkl", "wb") as f:
    pickle.dump(list(train_gen.class_indices.keys()), f)

print("✅ Đã train và lưu model + labels vào Drive")


Found 286 images belonging to 9 classes.
Found 69 images belonging to 9 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
10/18 ━━━━━━━━━━━━━━━━━━━━ 3s 476ms/step - accuracy: 0.1003 - loss: 39.4058

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 671ms/step - accuracy: 0.0929 - loss: 43.6641 - val_accuracy: 0.1159 - val_loss: 15.5283
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 531ms/step - accuracy: 0.1079 - loss: 28.6369 - val_accuracy: 0.1159 - val_loss: 4.1088
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 11s 547ms/step - accuracy: 0.1268 - loss: 10.3772 - val_accuracy: 0.1304 - val_loss: 2.3107
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 11s 601ms/step - accuracy: 0.1023 - loss: 2.2701 - val_accuracy: 0.1304 - val_loss: 2.1705
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 11s 599ms/step - accuracy: 0.1308 - loss: 2.2536 - val_accuracy: 0.1304 - val_loss: 2.2277
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 529ms/step - accuracy: 0.1225 - loss: 2.1915 - val_accuracy: 0.1304 - val_loss: 2.2249
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 11s 542ms/step - accuracy: 0.1552 - loss: 2.1671 - val_accuracy: 0.1304 - val_loss: 2.2219
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 11s 601ms/step - accuracy: 0.1286 - loss: 2.1694 - val_accuracy: 0

✅ Đã train và lưu model + labels vào Drive


In [ ]:
from tensorflow.keras.models import load_model
import pickle

model_money = load_model("/content/drive/MyDrive/money_ann_best.h5")

with open("/content/drive/MyDrive/money_labels.pkl", "rb") as f:
    labels_money = pickle.load(f)

print("✅ Đã load model và labels:", labels_money)


✅ Đã load model và labels: ['1000 VND', '10000 VND', '100000 VND', '2000 VND', '20000 VND', '200000 VND', '5000 VND', '50000 VND', '500000 VND']


In [ ]:
import numpy as np

def predict_money(img):
    img = img.resize(img_size)
    x = np.array(img)/255.0
    x = np.expand_dims(x, axis=0)
    preds = model_money.predict(x)
    return {labels_money[i]: float(preds[0][i]) for i in range(len(labels_money))}


In [ ]:
import os

feedback_dir = "/content/drive/MyDrive/money_feedback"
os.makedirs(feedback_dir, exist_ok=True)

def save_feedback(img, correct_label):
    if correct_label not in labels_money:
        return f"❌ Nhãn {correct_label} không hợp lệ. Các nhãn hợp lệ: {labels_money}"

    label_dir = os.path.join(feedback_dir, correct_label)
    os.makedirs(label_dir, exist_ok=True)
    img_path = os.path.join(label_dir, f"fb_{len(os.listdir(label_dir))}.jpg")
    img.save(img_path)
    return f"✅ Feedback đã lưu với nhãn {correct_label}"

def update_model():
    datagen_fb = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    fb_gen = datagen_fb.flow_from_directory(
        feedback_dir,
        target_size=img_size,
        batch_size=8,
        class_mode="categorical"
    )

    if fb_gen.samples > 0:
        model_money.fit(fb_gen, epochs=2)
        model_money.save("/content/drive/MyDrive/money_ann_best.h5")
        return "✅ Mô hình đã được cập nhật từ feedback."
    else:
        return "⚠️ Chưa có feedback để cập nhật."


In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## 💵 Nhận diện tiền Việt Nam + Feedback")

    with gr.Row():
        with gr.Column():
            img_input = gr.Image(type="pil", label="Upload ảnh tờ tiền")
            predict_btn = gr.Button("Dự đoán")
            output = gr.Label(num_top_classes=3)

        with gr.Column():
            correct_label = gr.Textbox(label="Nếu sai, nhập nhãn đúng (ví dụ: 10000 VND)")
            feedback_btn = gr.Button("Lưu Feedback")
            update_btn = gr.Button("Cập nhật mô hình")
            fb_msg = gr.Textbox(label="Thông báo")

    predict_btn.click(predict_money, inputs=img_input, outputs=output)
    feedback_btn.click(save_feedback, inputs=[img_input, correct_label], outputs=fb_msg)
    update_btn.click(lambda: update_model(), outputs=fb_msg)

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d62a61c18d194fd4a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Found 1 images belonging to 1 classes.
Epoch 1/2


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 667, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 349, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2274, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
